# Evaluación estadística

En este capítulo se evalúa la calidad de las imputaciones realizadas.  
Se utilizan métricas específicas para comparar los valores imputados frente a los verdaderos (cuando es posible).

**Puntos clave:**
- Definir métricas de error (MAE, RMSE) para variables numéricas.
- Calcular exactitud para variables categóricas.
- Comparar el desempeño de las distintas técnicas.
- Seleccionar la técnica más adecuada en función de las métricas obtenidas.

Esta evaluación aporta evidencia objetiva para justificar la técnica elegida.


In [5]:
# Separar columnas por tipo
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in df.columns if c not in num_cols]

print("Numéricas:", len(num_cols), "→", num_cols[:5], "...")
print("Categóricas:", len(cat_cols), "→", cat_cols[:5], "...")


Numéricas: 6 → ['edad', 'altura_cm', 'ingresos', 'gasto_mensual', 'puntuacion_credito'] ...
Categóricas: 6 → ['fecha', 'sexo', 'ciudad', 'nivel_educativo', 'segmento'] ...


La salida muestra la clasificación automática de las variables del dataset:

- **Numéricas (6):**  
  `edad`, `altura_cm`, `ingresos`, `gasto_mensual`, `puntuacion_credito`.  
  Son variables cuantitativas continuas o discretas que permiten cálculos estadísticos.

- **Categóricas (6):**  
  `fecha`, `sexo`, `ciudad`, `nivel_educativo`, `segmento`.  
  Son variables cualitativas o de identificación que agrupan registros en categorías.

 **Observación:**  
La variable `fecha` aparece clasificada como categórica, aunque en muchos análisis puede transformarse a tipo fecha para estudiar tendencias temporales.


In [16]:
# --- Evaluación de imputación: enmascaramiento + múltiples técnicas + métricas ---
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import numpy as np
import pandas as pd

# 1) Cargar datos (solo la base principal)
csv_path = Path.cwd().parent / "base_imputacion_mixta_1000.csv"
if not csv_path.exists():
    raise FileNotFoundError("No se encontró base_imputacion_mixta_1000.csv en la carpeta actual")

df = pd.read_csv(csv_path)
print("Archivo cargado:", csv_path.name, "| Dimensiones:", df.shape)

# 2) Detectar columnas tipo fecha y convertirlas a ordinal (para poder imputar)
date_cols = []
for c in df.columns:
    if df[c].dtype == "object":
        parsed = pd.to_datetime(df[c], errors="coerce")
        if parsed.notna().mean() > 0.8:   # si más del 80% son fechas válidas
            df[c] = parsed
            date_cols.append(c)
for c in date_cols:
    df[c] = df[c].map(lambda x: x.toordinal() if pd.notna(x) else np.nan)

# 3) Separar por tipo
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in df.columns if c not in num_cols]
print("Fechas convertidas:", date_cols)
print("Numéricas:", len(num_cols), "| Categóricas:", len(cat_cols))

# 4) Enmascaramiento: ocultar aleatoriamente un 10% de valores observados (mismo para todas las técnicas)
mask_rate = 0.10
rng = np.random.default_rng(42)

df_obs = df.copy()
df_masked = df_obs.copy()
mask_positions = {}

for c in num_cols:
    idx = df_obs[c].dropna().sample(frac=mask_rate, random_state=42).index
    mask_positions[c] = idx
    df_masked.loc[idx, c] = np.nan

for c in cat_cols:
    idx = df_obs[c].dropna().sample(frac=mask_rate, random_state=42).index
    mask_positions[c] = idx
    df_masked.loc[idx, c] = np.nan

# 5) Definir técnicas de imputación
from sklearn.experimental import enable_iterative_imputer  # noqa: F401
from sklearn.impute import IterativeImputer, SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# MICE (num) + Moda (cat)
pipe_mice = Pipeline([
    ("ct", ColumnTransformer([
        ("num", IterativeImputer(random_state=42, max_iter=10, sample_posterior=True), num_cols),
        ("cat", SimpleImputer(strategy="most_frequent"), cat_cols),
    ]))
])

# KNN (num) + Moda (cat)
pipe_knn = Pipeline([
    ("ct", ColumnTransformer([
        ("num", KNNImputer(n_neighbors=5, weights="distance"), num_cols),
        ("cat", SimpleImputer(strategy="most_frequent"), cat_cols),
    ]))
])

# Simple: Mediana (num) + Moda (cat)
pipe_simple = Pipeline([
    ("ct", ColumnTransformer([
        ("num", SimpleImputer(strategy="median"), num_cols),
        ("cat", SimpleImputer(strategy="most_frequent"), cat_cols),
    ]))
])

pipelines = {
    "MICE": pipe_mice,
    "KNN": pipe_knn,
    "Simple": pipe_simple,
}

# 6) Función de evaluación común (MAE, RMSE para numéricas; ACC para categóricas)
from sklearn.metrics import mean_absolute_error, mean_squared_error

def evaluar(df_masked, nombre, pipe):
    X_imp = pipe.fit_transform(df_masked)
    X_imp = pd.DataFrame(X_imp, columns=num_cols + cat_cols)

    num_mae, num_rmse = {}, {}
    for c in num_cols:
        idx = mask_positions.get(c, [])
        if len(idx) == 0:
            continue
        y_true = df_obs.loc[idx, c]
        y_pred = X_imp.loc[idx, c]
        num_mae[c]  = mean_absolute_error(y_true, y_pred)
        num_rmse[c] = np.sqrt(mean_squared_error(y_true, y_pred))

    cat_acc = {}
    for c in cat_cols:
        idx = mask_positions.get(c, [])
        if len(idx) == 0:
            continue
        y_true = df_obs.loc[idx, c].astype(str)
        y_pred = X_imp.loc[idx, c].astype(str)
        cat_acc[c] = (y_true == y_pred).mean()

    df_res = pd.DataFrame({
        f"{nombre}_MAE_num":  pd.Series(num_mae),
        f"{nombre}_RMSE_num": pd.Series(num_rmse),
        f"{nombre}_ACC_cat": pd.Series(cat_acc),
    })
    return df_res, X_imp

# 7) Ejecutar todas las técnicas y combinar resultados
resultados_list = []
imputados_dict = {}  # por si luego quieres graficar/seguir usando los datasets imputados

for nombre, pipe in pipelines.items():
    res, X_imp = evaluar(df_masked, nombre, pipe)
    resultados_list.append(res)
    imputados_dict[nombre] = X_imp
    print(f"✓ Técnica evaluada: {nombre}")

# Combinar por índice de variable
resultados = pd.concat(resultados_list, axis=1).sort_index()
resultados





Archivo cargado: base_imputacion_mixta_1000.csv | Dimensiones: (1000, 12)
Fechas convertidas: ['fecha']
Numéricas: 7 | Categóricas: 5
✓ Técnica evaluada: MICE
✓ Técnica evaluada: KNN
✓ Técnica evaluada: Simple


,MICE_MAE_num,MICE_RMSE_num,MICE_ACC_cat,KNN_MAE_num,KNN_RMSE_num,KNN_ACC_cat,Simple_MAE_num,Simple_RMSE_num,Simple_ACC_cat
altura_cm,338.908404,414.610503,NaN,8.049474,10.361854,NaN,6.944184,9.130546,NaN
ciudad,NaN,NaN,0.431579,NaN,NaN,0.431579,NaN,NaN,0.431579
demanda,1116.155362,1369.556298,NaN,19.892738,24.829851,NaN,21.220320,24.689903,NaN
edad,583.243612,697.333786,NaN,13.673262,16.427839,NaN,12.195876,14.319261,NaN
estado_civil,NaN,NaN,0.400000,NaN,NaN,0.400000,NaN,NaN,0.400000
fecha,338.989417,407.694070,NaN,252.870156,309.416954,NaN,247.630000,286.071058,NaN
gasto_mensual,1027.012966,1289.865163,NaN,521.104058,670.132055,NaN,506.048470,625.892501,NaN
ingresos,67286.853933,87144.533853,NaN,969.605198,1226.638077,NaN,1031.735908,1271.022250,NaN
nivel_educativo,NaN,NaN,0.400000,NaN,NaN,0.400000,NaN,NaN,0.400000
puntuacion_credito,840.435995,1057.603370,NaN,72.182550,88.033834,NaN,71.242636,86.124923,NaN


# Resultados de la evaluación de imputación

La tabla compara tres técnicas: **MICE**, **KNN** y **Simple** (mediana/moda), evaluadas con métricas en variables numéricas (**MAE, RMSE**) y categóricas (**ACC**).

## Principales hallazgos

- **Numéricas:**  
  - **MICE** presenta errores muy altos en varias variables → mal desempeño.  
  - **KNN** y **Simple** muestran errores bajos y consistentes → son más confiables.

- **Categóricas:**  
  - La exactitud (**ACC**) se mantiene entre **0.40 y 0.56**, lo que indica que imputar categorías es más difícil.  
  - **KNN** y **Simple** tienen resultados similares.

## Conclusión

- Para **numéricas**, conviene usar **KNN o Simple**, ya que superan ampliamente a MICE.  
- Para **categóricas**, la moda (Simple) es suficiente; KNN no aporta gran ventaja.  
- **MICE** podría mejorarse ajustando parámetros o preprocesando los datos, pero en este caso no es recomendable.

